<a href="https://colab.research.google.com/github/zanasaed/NoteBook/blob/main/ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Links of library and tutorial 

# definition 


## Key Concepts
**Node**

It refers to a single running instance of Elasticsearch. Single physical and virtual server accommodates multiple nodes depending upon the capabilities of their physical resources like RAM, storage and processing power.

**Cluster**

It is a collection of one or more nodes. Cluster provides collective indexing and search capabilities across all the nodes for entire data.

**Index**

It is a collection of different type of documents and their properties. Index also uses the concept of shards to improve the performance. For example, a set of document contains data of a social networking application.

**Document**

It is a collection of fields in a specific manner defined in JSON format. Every document belongs to a type and resides inside an index. Every document is associated with a unique identifier called the UID.

**Shard**

Indexes are horizontally subdivided into shards. This means each shard contains all the properties of document but contains less number of JSON objects than index. The horizontal separation makes shard an independent node, which can be store in any node. Primary shard is the original horizontal part of an index and then these primary shards are replicated into replica shards.

**Replicas**

Elasticsearch allows a user to create replicas of their indexes and shards. Replication not only helps in increasing the availability of data in case of failure, but also improves the performance of searching by carrying out a parallel search operation in these replicas.




# Search

## **query lite** 
for search simple and fast we can usae this format query in web browser 


INDEX/TYPE/_search?q=title:str

movies/move/_search?q=+year:>2010+title:trek


example
http://localhost:9200/digikala/seller/_search?q=kind:seller_data


The problem is when you're sending across urls you need to encode them very often and that can make things very confusing very quickly.


So you couldn't actually reliably send across that string as is as part of a rest request you would have to url rncoded it in order to securely transmit it over the Internet. And that means that any special characters need to be encoded into their hexadecimal equivalency or encoded out.


If you want learn more just search for URI  search.
That's what's formally known these days and elasticsearch.
If you go to elasticsearch Web site on elastic.co and search for URI search you'll see the
full documentation on how to construct these things.

